In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../notion_data/원달러환율수익률.csv')

In [4]:
df.head()

,Price,Close,High,Low,Open,Volume,환율수익률(%)
0,2024-12-30,1472.800049,1475.250000,1466.359985,1472.800049,0,0.396053
1,2024-12-27,1466.989990,1487.040039,1462.800049,1466.989990,0,0.643520
2,2024-12-26,1457.609985,1470.290039,1454.770020,1457.609985,0,0.288282
3,2024-12-25,1453.420044,1458.079956,1448.449951,1453.420044,0,0.743058
4,2024-12-24,1442.699951,1459.920044,1449.329956,1442.699951,0,-0.188187


In [5]:
df2 = df.copy()

In [6]:
df = df[['Price', 'Close']]

In [7]:
df.rename(columns={'Price' : '날짜',
                   'Close' : '원자료'}, inplace= True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2605 entries, 0 to 2604
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      2605 non-null   object 
 1   원자료     2605 non-null   float64
dtypes: float64(1), object(1)
memory usage: 40.8+ KB


In [9]:
df.head()

,날짜,원자료
0,2024-12-30,1472.800049
1,2024-12-27,1466.989990
2,2024-12-26,1457.609985
3,2024-12-25,1453.420044
4,2024-12-24,1442.699951


In [10]:
df = df.sort_values('날짜').reset_index(drop=True)


In [11]:
df.head()

,날짜,원자료
0,2015-01-01,1092.699951
1,2015-01-02,1093.599976
2,2015-01-05,1111.000000
3,2015-01-06,1108.500000
4,2015-01-07,1097.300049


In [12]:
import pandas as pd

# 예시 DataFrame (여기선 df가 이미 정의되어 있다고 가정)
print("변환 전 '원자료' 컬럼 타입:")
print(df['원자료'].dtype)

print("\n변환 전 '원자료' 컬럼 내용:")
print(df['원자료'])

# 1. 문자열로 변환한 후 쉼표 제거
df['원자료'] = df['원자료'].astype(str).str.replace(',', '', regex=False)

# 2. 숫자로 변환 (변환 불가능한 값은 NaN 처리)
df['원자료'] = pd.to_numeric(df['원자료'], errors='coerce')

print("\n--- 변환 후 ---")
print("변환 후 '원자료' 컬럼 타입:")
print(df['원자료'].dtype)

print("\n변환 후 '원자료' 컬럼 내용:")
print(df['원자료'])


변환 전 '원자료' 컬럼 타입:
float64

변환 전 '원자료' 컬럼 내용:
0       1092.699951
1       1093.599976
2       1111.000000
3       1108.500000
4       1097.300049
           ...     
2600    1442.699951
2601    1453.420044
2602    1457.609985
2603    1466.989990
2604    1472.800049
Name: 원자료, Length: 2605, dtype: float64

--- 변환 후 ---
변환 후 '원자료' 컬럼 타입:
float64

변환 후 '원자료' 컬럼 내용:
0       1092.699951
1       1093.599976
2       1111.000000
3       1108.500000
4       1097.300049
           ...     
2600    1442.699951
2601    1453.420044
2602    1457.609985
2603    1466.989990
2604    1472.800049
Name: 원자료, Length: 2605, dtype: float64


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2605 entries, 0 to 2604
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      2605 non-null   object 
 1   원자료     2605 non-null   float64
dtypes: float64(1), object(1)
memory usage: 40.8+ KB


In [14]:
df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce')


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2605 entries, 0 to 2604
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   날짜      2605 non-null   datetime64[ns]
 1   원자료     2605 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 40.8 KB


In [16]:
# '날짜'를 인덱스로 설정
df = df.set_index('날짜')

# 일별 수익률 계산 (pct_change는 전일 대비 퍼센트 변동)
returns = df.pct_change()

# 결과 저장용 DataFrame 생성
result = pd.DataFrame(index=df.index)

# (shift, 라벨) 쌍 정의: 
# shift=1 → -1~-5일, shift=6 → -6~-10일, shift=11 → -11~-15일, shift=16 → -16~-20일
windows = [
    (1, '(-1~-5)'),
    (6, '(-6~-10)'),
    (11, '(-11~-15)'),
    (16, '(-16~-20)')
]

# 각 컬럼에 대해 네 구간별 표준편차 계산
for col in df.columns:
    for shift, label in windows:
        result[f"{col}{label}"] = (
            returns[col]
            .shift(shift)              # 과거 shift 일만큼 뒤로
            .rolling(window=5)         # 5일간
            .std()                     # 표준편차
        )

# 인덱스를 '날짜' 컬럼으로 되돌리고
result = result.reset_index()

# 결과 확인
print(result.head())
# 필요시 to_csv 등으로 저장



          날짜  원자료(-1~-5)  원자료(-6~-10)  원자료(-11~-15)  원자료(-16~-20)
0 2015-01-01         NaN          NaN           NaN           NaN
1 2015-01-02         NaN          NaN           NaN           NaN
2 2015-01-05         NaN          NaN           NaN           NaN
3 2015-01-06         NaN          NaN           NaN           NaN
4 2015-01-07         NaN          NaN           NaN           NaN


In [17]:
result.head(30)

,날짜,원자료(-1~-5),원자료(-6~-10),원자료(-11~-15),원자료(-16~-20)
0,2015-01-01,NaN,NaN,NaN,NaN
1,2015-01-02,NaN,NaN,NaN,NaN
2,2015-01-05,NaN,NaN,NaN,NaN
3,2015-01-06,NaN,NaN,NaN,NaN
4,2015-01-07,NaN,NaN,NaN,NaN
5,2015-01-08,NaN,NaN,NaN,NaN
6,2015-01-09,0.009441,NaN,NaN,NaN
7,2015-01-12,0.009625,NaN,NaN,NaN
8,2015-01-13,0.005183,NaN,NaN,NaN
9,2015-01-14,0.005517,NaN,NaN,NaN


In [18]:
result = result.rename(columns={
    '원자료(-1~-5)' : '환율수익률표준편차(-1~-5)',
    '원자료(-6~-10)' : '환율수익률표준편차(-6~-10)',
    '원자료(-11~-15)' : '환율수익률표준편차(-11~-15)',
    '원자료(-16~-20)' : '환율수익률표준편차(-16~-20)',
    '기타' : '기타'
})

In [19]:
macro_df = pd.read_csv('../data/macro_data.csv')

In [20]:
macro_df.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '코스닥수익률표준편차(-11~-15)', '코스닥수익률표준편차(-16~-20)', '코스피수익률표준편차(-1~-5)',
       '코스피수익률표준편차(-6~-10)', '코스피수익률표준편차(-11~-15)', '코스피수익률표준편차(-16~-20)',
       '나스닥수익률표준편차(-1~-5)', '나스닥수익률표준편차(-6~-10)', '나스닥수익률표준편차(-11~-15)',
       '나스닥수익률표준편차(-16~-20)'],
      dtype='object', length=168)

In [21]:
# 1) Convert both columns to datetime
macro_df['상장일'] = pd.to_datetime(macro_df['상장일'])
result['날짜'] = pd.to_datetime(result['날짜'])

# 2) Sort both DataFrames by their date keys
merged_full = macro_df.sort_values('상장일').reset_index(drop=True)
result = result.sort_values('날짜').reset_index(drop=True)

merged_full = pd.merge_asof(
    merged_full,
    result,
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

In [22]:
merged_full.columns

Index(['회사명', 'stock_code', '상장일', 'kospi200(-20)', 'nasdaq(-20)',
       'putcall(-20)', 'VIXCLS', '미국10년물국채금리수익률(-20)', '시장금리국고채3년',
       '환율변동률(-20)',
       ...
       '코스피수익률표준편차(-11~-15)', '코스피수익률표준편차(-16~-20)', '나스닥수익률표준편차(-1~-5)',
       '나스닥수익률표준편차(-6~-10)', '나스닥수익률표준편차(-11~-15)', '나스닥수익률표준편차(-16~-20)',
       '환율수익률표준편차(-1~-5)', '환율수익률표준편차(-6~-10)', '환율수익률표준편차(-11~-15)',
       '환율수익률표준편차(-16~-20)'],
      dtype='object', length=172)

In [23]:
merged_full.to_csv('macro_data.csv', encoding='utf-8-sig', index=False)
